# you.com News API <> Llama Index

DISCLAIMER: This notebook requires the latest version of LlamaIndex to be installed from this [branch](https://github.com/run-llama/llama_index/pull/13934).  The cells in this notebook will not successfully run if the latest version of LlamaIndex is not being utilized.

Creating a chatbot that can provide personalized news on any topic using you.com news API involves several steps, including obtaining the necessary credentials, and programming your chatbot to use the API. 

## Install required packages

In [1]:
%%capture
! pip install llama-index-retrievers-you==0.1.2
! pip install dotenv==0.0.5
! pip install python-dotenv==1.0.1

## Load API keys

In [2]:
# assumes a .env file exists with api keys YDC_API_KEY and OPENAI_API_KEY

from dotenv import load_dotenv
import os

load_dotenv()

True

## Create a retriever

In [3]:
# TODO update the pinned version of llama-index-retrievers-you after PR is merged: https://github.com/run-llama/llama_index/pull/13934
from llama_index.retrievers.you import YouRetriever

retriever = YouRetriever(endpoint_type="news")
retrieved_results = retriever.retrieve("national parks in the US")

print(retrieved_results[0].get_content())

TypeError: YouRetriever.__init__() got an unexpected keyword argument 'endpoint_type'

# Create a chat engine

Conceptually, it is a stateful analogy of a Query Engine. 

In [26]:
from llama_index.core.chat_engine.context import ContextChatEngine

chat_engine = ContextChatEngine.from_defaults(retriever=retriever)
response = chat_engine.chat("Canada")

print(response)

NameError: name 'retriever' is not defined

### Include a timeframe

In [27]:
from llama_index.core.chat_engine.context import ContextChatEngine

chat_engine = ContextChatEngine.from_defaults(retriever=retriever)
response = chat_engine.chat("Norway in Jan 2024")

print(response)

NameError: name 'retriever' is not defined

# Create a chat engine with memory

By keeping track of the conversation history, it can answer questions with past context in mind.
 
`condense_plus_context` - A combination of condense_question and context. Look at the chat history and re-write the user message to be a retrieval query for the index. The retrieved text is inserted into the system prompt, so that the chat engine can either respond naturally or use the context from the query engine.

In [28]:
from llama_index.core.memory import ChatMemoryBuffer
from llama_index.core.chat_engine import CondensePlusContextChatEngine
from llama_index.llms.openai import OpenAI

memory = ChatMemoryBuffer.from_defaults(token_limit=3900)
llm = OpenAI(model="gpt-4o")

chat_engine = CondensePlusContextChatEngine.from_defaults(
    retriever=retriever,
    memory=memory,
    llm=llm,
)

ModuleNotFoundError: No module named 'llama_index.llms'

In [29]:
# relevant news article with a summary of the news
response = chat_engine.chat("Salesforce")
print(response)

NameError: name 'chat_engine' is not defined

In [30]:
# uses memory to relate the Hubspot query to market performance news in the query above
response = chat_engine.chat("Hubspot")
print(response)

NameError: name 'chat_engine' is not defined

## Personalize news

Using a custom prompt, this news feed can be customized to specific threads, such as this one focused on politics and economics.

In [31]:
from llama_index.core.memory import ChatMemoryBuffer
from llama_index.core.chat_engine import CondensePlusContextChatEngine
from llama_index.core import PromptTemplate

custom_prompt = PromptTemplate(
    """\
You're a top-tier reporter focused on news about {question}.\n
YOU SHOULD ALWAYS RESEARCH THE FOLLOWING QUERIES: \n
1. {question} recent news and latest development\n
2. recent regulatory changes affecting {question}\n
3. recent economic events impacting {question}\n
"""
)

memory = ChatMemoryBuffer.from_defaults(token_limit=3900)
llm = OpenAI(model="gpt-4o")

chat_engine = CondensePlusContextChatEngine.from_defaults(
    retriever=retriever,
    condense_question_prompt=custom_prompt,
    memory=memory,
    llm=llm,
)

NameError: name 'OpenAI' is not defined

In [32]:
response = chat_engine.chat("Mongolia")
print(response)

NameError: name 'chat_engine' is not defined